In [1]:
%load_ext Cython
import numpy as np
import scipy.ndimage
import scipy.signal

In [10]:
%%cython

cpdef int max_pooling_batch(double [:,:,:,:] src, double [:,:,:,:] dst, int [:,:,:,:] max_ind, 
                            int win_h, int win_w, int stride_h, int stride_w):
    cdef:
        int num = src.shape[0]
        int ch = src.shape[1]
        int h_src = src.shape[2]
        int w_src = src.shape[3]
        int h_dst = dst.shape[2]
        int w_dst = dst.shape[3]
        int i, j, k, l, m, n
        int dst_ind = 0, src_ind = 0
        double *src_ptr = &src[0][0][0][0]
        double *dst_ptr = &dst[0][0][0][0]
        int *ind_ptr = &max_ind[0][0][0][0]
        double max_val
        int current_max_ind
        double tmp
        
    for i in range(num):
        for j in range(ch):
            for k in range(0, h_src, stride_h):
                for l in range(0, w_src, stride_w):
                    print k
                    print l
                    src_ind = i * ch * h_src * w_src + j * h_src * w_src + k * w_src + l
                    max_val = -1e20
                    for m in range(win_h):
                        for n in range(win_w):
                            tmp = src_ptr[src_ind + n]
                            if tmp > max_val:
                                max_val = tmp
                                current_max_ind = src_ind + n
                        src_ind += w_src
                    dst_ptr[dst_ind] = max_val
                    ind_ptr[dst_ind] = current_max_ind
                    dst_ind += 1
    return 0

cpdef int backprop_for_max_pooling(double [:,:,:,:] src, double [:,:,:,:] dst, int [:,:,:,:] max_ind):
    cdef:
        int num = src.shape[0]
        int ch = src.shape[1]
        int h_src = src.shape[2]
        int w_src = src.shape[3]
        int h_dst = dst.shape[2]
        int w_dst = dst.shape[3]
        int i
        double *src_ptr = &src[0][0][0][0]
        double *dst_ptr = &dst[0][0][0][0]
        int *ind_ptr = &max_ind[0][0][0][0]
        int total_dst = num * ch * h_dst * w_dst
    
    for i in range(total_dst):
        src_ptr[ind_ptr[i]] += dst_ptr[i]
    
# cpdef int get_mean_index(int src_h, int src_w, int win_h, int win_w, 
#                          int stride_h, int stride_w, int [:,:] mean_ind):
#     return 0

# cpdef int mean_pooling_batch(double [:,:,:,:] src, double [:,:,:,:] dst,
#                              int win_h, int win_w, int stride_h, int stride_w):
#     return 0

 

In [23]:
a = np.array([[[[1, 2, 0, 0],
              [5, 3, 0, 4],
              [0, 0, 0, 7],
              [9, 3, 0, 0]],
              
              [[9, 8, 0, 0],
              [1, 4, 1, 1],
              [6, 0, 4, 3],
              [0, 8, 0, 0]]],
             
             [[[1, 2, 0, 0],
              [5, 3, 0, 4],
              [0, 0, 0, 7],
              [9, 3, 0, 0]],
              
              [[9, 8, 0, 0],
              [1, 4, 1, 1],
              [6, 0, 4, 3],
              [0, 8, 0, 0]]],
             
             
             ], dtype=float)
max_ind = np.zeros((2, 2, 1, 1), dtype='int32')
result =  np.zeros((2, 2, 1, 1))
print max_pooling_batch(a, result, max_ind, 3, 3, 2, 2)
print result
print max_ind

# a = np.random.random((100, 3, 128, 128))
# out = np.zeros((100, 3, 32, 32))
# ind = np.zeros((100, 3, 32, 32), dtype='int32')
# %timeit max_pooling_batch(a, out, ind, 4, 4, 4, 4)

0
[[[[ 5.]]

  [[ 9.]]]


 [[[ 9.]]

  [[ 9.]]]]
[[[[ 4]]

  [[12]]]


 [[[12]]

  [[12]]]]
